In [117]:
import torch
import os
import json
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)
from platform import python_version
print(python_version())
from collections import defaultdict
from typing import Any, Iterable, List, Optional, Tuple, Union
from torch import Tensor
from torch_geometric.utils import to_dense_adj
from torch_geometric.loader import DataLoader

# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv
from torch_geometric.nn import RGCNConv
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

1.13.1+cu116
3.8.10


In [118]:
# here is the modified to_networkx function that doesn't throw exceptions

def from_networkx(
    G: Any,
    group_node_attrs: Optional[Union[List[str], all]] = None,
    group_edge_attrs: Optional[Union[List[str], all]] = None,
) -> 'torch_geometric.data.Data':
    r"""Converts a :obj:`networkx.Graph` or :obj:`networkx.DiGraph` to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        G (networkx.Graph or networkx.DiGraph): A networkx graph.
        group_node_attrs (List[str] or all, optional): The node attributes to
            be concatenated and added to :obj:`data.x`. (default: :obj:`None`)
        group_edge_attrs (List[str] or all, optional): The edge attributes to
            be concatenated and added to :obj:`data.edge_attr`.
            (default: :obj:`None`)

    .. note::

        All :attr:`group_node_attrs` and :attr:`group_edge_attrs` values must
        be numeric.

    Examples:

        >>> edge_index = torch.tensor([
        ...     [0, 1, 1, 2, 2, 3],
        ...     [1, 0, 2, 1, 3, 2],
        ... ])
        >>> data = Data(edge_index=edge_index, num_nodes=4)
        >>> g = to_networkx(data)
        >>> # A `Data` object is returned
        >>> from_networkx(g)
        Data(edge_index=[2, 6], num_nodes=4)
    """
    import networkx as nx

    from torch_geometric.data import Data

    G = nx.convert_node_labels_to_integers(G)
    G = G.to_directed() if not nx.is_directed(G) else G

    if isinstance(G, (nx.MultiGraph, nx.MultiDiGraph)):
        edges = list(G.edges(keys=False))
    else:
        edges = list(G.edges)

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    data = defaultdict(list)

    if G.number_of_nodes() > 0:
        node_attrs = list(next(iter(G.nodes(data=True)))[-1].keys())
    else:
        node_attrs = {}

    if G.number_of_edges() > 0:
        edge_attrs = list(next(iter(G.edges(data=True)))[-1].keys())
    else:
        edge_attrs = {}

    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        if set(feat_dict.keys()) != set(node_attrs):
            raise ValueError('Not all nodes contain the same attributes')
        for key, value in feat_dict.items():
            data[str(key)].append(value)

    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        if set(feat_dict.keys()) != set(edge_attrs):
            raise ValueError('Not all edges contain the same attributes')
        for key, value in feat_dict.items():
            key = f'edge_{key}' if key in node_attrs else key
            data[str(key)].append(value)

    for key, value in G.graph.items():
        key = f'graph_{key}' if key in node_attrs else key
        data[str(key)] = value

    for key, value in data.items():
        if isinstance(value, (tuple, list)) and isinstance(value[0], Tensor):
            data[key] = torch.stack(value, dim=0)
        else:
            try:
                data[key] = torch.tensor(value)
            except:
                pass

    data['edge_index'] = edge_index.view(2, -1)
    data = Data.from_dict(data)

    if group_node_attrs is all:
        group_node_attrs = list(node_attrs)
    if group_node_attrs is not None:
        xs = []
        for key in group_node_attrs:
            x = data[key]
            x = x.view(-1, 1) if x.dim() <= 1 else x
            xs.append(x)
            del data[key]
        data.x = torch.cat(xs, dim=-1)

    if group_edge_attrs is all:
        group_edge_attrs = list(edge_attrs)
    if group_edge_attrs is not None:
        xs = []
        for key in group_edge_attrs:
            key = f'edge_{key}' if key in node_attrs else key
            x = data[key]
            x = x.view(-1, 1) if x.dim() <= 1 else x
            xs.append(x)
            del data[key]
        data.edge_attr = torch.cat(xs, dim=-1)

    if data.x is None and data.pos is None:
        data.num_nodes = G.number_of_nodes()

    return data

In [119]:
from torch_geometric.typing import SparseTensor

def to_edge_index(adj: Union[Tensor, SparseTensor]) -> Tuple[Tensor, Tensor]:
    r"""Converts a :class:`torch.sparse.Tensor` or a
    :class:`torch_sparse.SparseTensor` to edge indices and edge attributes.

    Args:
        adj (torch.sparse.Tensor or SparseTensor): The adjacency matrix.

    :rtype: (:class:`LongTensor`, :class:`Tensor`)

    Example:

        >>> edge_index = torch.tensor([[0, 1, 1, 2, 2, 3],
        ...                            [1, 0, 2, 1, 3, 2]])
        >>> adj = to_torch_coo_tensor(edge_index)
        >>> to_edge_index(adj)
        (tensor([[0, 1, 1, 2, 2, 3],
                [1, 0, 2, 1, 3, 2]]),
        tensor([1., 1., 1., 1., 1., 1.]))
    """
    if isinstance(adj, SparseTensor):
        row, col, value = adj.coo()
        if value is None:
            value = torch.ones(row.size(0), device=row.device)
        return torch.stack([row, col], dim=0), value

    if adj.requires_grad:
        # Calling adj._values() will return a detached tensor.
        # Use `adj.coalesce().values()` instead to track gradients.
        adj = adj.coalesce()
        return adj.indices(), adj.values()

    return adj._indices(), adj._values()

In [120]:
label_dict = {
        "Point": 0,
        "Line": 1,
        "Circle": 2,
        "Ellipse": 3,
        "Spline": 4,
        "Conic": 5,
        "Arc": 6,
        "External": 7,
        "Stop": 8,
        "Unknown": 9,
        "SN_Start": 11,
        "SN_End": 12,
        "SN_Center": 13
    }

In [121]:
from torch_geometric.utils import degree
def get_sketch_features(graph, feature_dim):
    x = torch.zeros([graph.num_nodes, feature_dim])
    for idx, p in enumerate(graph.parameters):
        
        # convert label text into a feature value
        x[idx, 0] = label_dict[graph.label[idx]]/7
        
        param_dict = json.loads(p)
        for i, k in enumerate(param_dict.keys()):
            
            if i+2 == feature_dim:
                break
            
            # convert each parameter value into a feature value
            x[idx, i+1] = float(param_dict[k])
        
        x[idx, -1] = degree(graph.edge_index[0], graph.num_nodes)[idx]
        #print(idx, p)
        #print(x[idx])
    return x

In [122]:
def get_sketch_attr_y(graph):
    y = torch.zeros([graph.num_nodes, 1], dtype=torch.int64)
    #rint(graph.label)
    
    
    for i, l in enumerate(graph.label):
        y[i, 0] = label_dict[l]
    
    return y

In [123]:
def get_sketch_adj(graph):
    tst = T.ToSparseTensor()
    return tst(graph).adj_t

In [124]:
def find_large_graph(graphs):
    bestN = 0
    bestI = 0
    for i, g in enumerate(graphs):
        if len(g) > bestN:
            bestN = len(g)
            bestI = i
    #print(bestI)
    return graphs[bestI]

In [125]:
# custom dataset class of custom attributes
from torch_geometric.data import Dataset
class SketchgraphDataset(Dataset):
    def __init__(self, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(transform, pre_transform, pre_filter)
        
        self.data = []
        seq_data = flat_array.load_dictionary_flat('datasets/sg_t16_validation.npy')
        print(len(seq_data['sequences']))

        #test_graph_seq = find_large_graph(seq_data['sequences'])
        test_graph_seq = seq_data['sequences'][206778]
        test_graph_seq1 = seq_data['sequences'][10]

        sketchgraps_list = seq_data['sequences'][0:1023]

        for sg in sketchgraps_list:
            #print(test_graph_seq)

            # convert first to pyGraphViz graph using sketchgraph's function
            pgv_graph = sketchgraphs.data.sequence.pgvgraph_from_sequence(sg)
            #print(pgv_graph)

            # then to networkx graph
            nx_graph = nx.Graph(pgv_graph)
            #print(nx_graph)

            # finally to pyTorch graph
            graph = from_networkx(nx_graph)

            # next we need to add required attributes: x, y, adj_t
            graph.x = get_sketch_features(graph, 8)
            graph.y = get_sketch_attr_y(graph)
            graph.adj_t = get_sketch_adj(graph)
            #print(graph.x[0])
            graph.edge_index = to_edge_index(graph.adj_t)[0]
            if not hasattr(graph, 'edge_label'):
                continue
            #print(len(graph.edge_label))
            self.data.append(graph)
            
    def len(self):
        return len(self.data)
    
    def get(self, idx):
        return self.data[idx]
    
"""def custom_collate(batch, b):
    print("AAAAA")
    edge_label_batch = []
    for b in batch:
        #print(b.edge_label)
        edge_label_batch.append(b.edge_label)
    return edge_label_batch"""
    

'def custom_collate(batch, b):\n    print("AAAAA")\n    edge_label_batch = []\n    for b in batch:\n        #print(b.edge_label)\n        edge_label_batch.append(b.edge_label)\n    return edge_label_batch'

In [126]:
import sketchgraphs
import networkx as nx
from sketchgraphs.data import flat_array

dataset = SketchgraphDataset()
    
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

for b in iter(data_loader):
    print(b)
#print(graph)
#print(graph.x)
#print(graph.y)

315228
DataBatch(
  index=[64],
  label=[64],
  parameters=[64],
  edge_label=[64],
  edge_parameters=[64],
  graph={},
  node={
    index=[64],
    label=[64],
    parameters=[64]
  },
  edge={
    index=[64],
    label=[64],
    parameters=[64]
  },
  num_nodes=819,
  x=[819, 8],
  y=[819, 1],
  adj_t=[819, 819, nnz=1984],
  edge_index=[2, 1984],
  batch=[819],
  ptr=[65]
)
DataBatch(
  index=[64],
  label=[64],
  parameters=[64],
  edge_label=[64],
  edge_parameters=[64],
  graph={},
  node={
    index=[64],
    label=[64],
    parameters=[64]
  },
  edge={
    index=[64],
    label=[64],
    parameters=[64]
  },
  num_nodes=954,
  x=[954, 8],
  y=[954, 1],
  adj_t=[954, 954, nnz=2401],
  edge_index=[2, 2401],
  batch=[954],
  ptr=[65]
)
DataBatch(
  index=[64],
  label=[64],
  parameters=[64],
  edge_label=[64],
  edge_parameters=[64],
  graph={},
  node={
    index=[64],
    label=[64],
    parameters=[64]
  },
  edge={
    index=[64],
    label=[64],
    parameters=[64]
  },
  nu

In [127]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    dataset_name = 'ogbn-arxiv'
    dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())
    data = dataset[0]
    print(data)
    #data = batch
    


    # Make the adjacency matrix to symmetric
    data.adj_t = data.adj_t.to_symmetric()
    #print(data.y)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # If you use GPU, the device should be cuda
    print('Device: {}'.format(device))
    data = data.to(device)
    split_idx = dataset.get_idx_split()
    print(split_idx['train'])
    #train_idx = torch.LongTensor(range(0, data.num_nodes)).to(device)
    #print(train_idx)

Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])
Device: cuda
tensor([     0,      1,      2,  ..., 169145, 169148, 169251])


In [128]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # TODO: Implement a function that initializes self.convs, 
        # self.bns, and self.softmax.

        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = None

        # A list of 1D batch normalization layers
        self.bns = None

        # The log softmax layer
        self.softmax = None

        ############# Your code here ############
        ## Note:
        ## 1. You should use torch.nn.ModuleList for self.convs and self.bns
        ## 2. self.convs has num_layers GCNConv layers
        ## 3. self.bns has num_layers - 1 BatchNorm1d layers
        ## 4. You should use torch.nn.LogSoftmax for self.softmax
        ## 5. The parameters you can set for GCNConv include 'in_channels' and 
        ## 'out_channels'. For more information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 6. The only parameter you need to set for BatchNorm1d is 'num_features'
        ## For more information please refer to the documentation: 
        ## https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        ## (~10 lines of code)

        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        self.softmax = torch.nn.LogSoftmax(dim=-1)

        self.convs.append(GCNConv(input_dim, hidden_dim))
        for i in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.linear = torch.nn.Linear(hidden_dim, output_dim)
        for i in range(num_layers - 1):
            self.bns.append(torch.nn.BatchNorm1d(hidden_dim))
            
        
        #########################################

        # Probability of an element getting zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, edge_index):
        # TODO: Implement a function that takes the feature tensor x and
        # edge_index tensor adj_t and returns the output tensor as
        # shown in the figure.

        out = None

        ############# Your code here ############
        ## Note:
        ## 1. Construct the network as shown in the figure
        ## 2. torch.nn.functional.relu and torch.nn.functional.dropout are useful
        ## For more information please refer to the documentation:
        ## https://pytorch.org/docs/stable/nn.functional.html
        ## 3. Don't forget to set F.dropout training to self.training
        ## 4. If return_embeds is True, then skip the last softmax layer
        ## (~7 lines of code)
        F.dropout.training = self.training
        
        for i in range(len(self.convs) - 1):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x)
        x = self.convs[-1](x, edge_index)
        x = self.linear(x)
        if self.return_embeds == False:
            x = self.softmax(x)
        out = x
        #########################################

        return out

In [129]:
def train(model, loader, train_idx, optimizer, loss_fn):
    # TODO: Implement a function that trains the model by 
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    ############# Your code here ############
    ## Note:
    ## 1. Zero grad the optimizer
    ## 2. Feed the data into the model
    ## 3. Slice the model output and label by train_idx
    ## 4. Feed the sliced output and label to loss_fn
    ## (~4 lines of code)
    for batch in iter(loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        o = model(batch.x, batch.edge_index)
        # o = o[train_idx] # we train on the whole graphs now
        loss = loss_fn(o, batch.y.squeeze(1))
        #########################################

        loss.backward()
        optimizer.step()

    return loss.item()

In [130]:
# Test function here
@torch.no_grad()
def test(model, loader, split_idx, evaluator, save_model_results=False):
    # TODO: Implement a function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None
    train_acc = 0
    valid_acc = 0
    test_acc = 0
    
    count = 0
    
    for batch in loader:
        count+=1
        batch = batch.to(device)
        ############# Your code here ############
        ## (~1 line of code)
        ## Note:
        ## 1. No index slicing here
        out = model(batch.x, batch.edge_index)
        #########################################

        y_pred = out.argmax(dim=-1, keepdim=True)
        
        #if count == 1:
            #print(y_pred, batch.y)
        
        train_acc += evaluator.eval({
            'y_true': batch.y,
            'y_pred': y_pred,
        })['acc']
        valid_acc += evaluator.eval({
            'y_true': batch.y,
            'y_pred': y_pred,
        })['acc']
        test_acc += evaluator.eval({
            'y_true': batch.y,
            'y_pred': y_pred,
        })['acc']

    train_acc /= count
    valid_acc /= count
    test_acc /= count

    if save_model_results:
        print ("Saving Model Predictions")

        data = {}
        data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

        df = pd.DataFrame(data=data)
        # Save locally as csv
        df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc

In [131]:
# Please do not change the args
if 'IS_GRADESCOPE_ENV' not in os.environ:
    args = {
      'device': device,
      'num_layers': 3,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 100,
    }
    args

In [132]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    model = GCN(8, args['hidden_dim'],
              dataset.num_classes, args['num_layers'],
              args['dropout']).to(device)
    evaluator = Evaluator(name='ogbn-arxiv')

In [133]:
# Please do not change these args
# Training should take <10min using GPU runtime
import copy
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # reset the parameters to initial random value
    model.reset_parameters()

    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
    loss_fn = F.nll_loss

    best_model = None
    best_valid_acc = 0

    for epoch in range(1, 1 + args["epochs"]):
        loss = train(model, data_loader, [], optimizer, loss_fn)
        result = test(model, data_loader, [], evaluator)
        train_acc, valid_acc, test_acc = result
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            best_model = copy.deepcopy(model)
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 1.7319, Train: 27.70%, Valid: 27.70% Test: 27.70%
Epoch: 02, Loss: 1.3135, Train: 30.91%, Valid: 30.91% Test: 30.91%
Epoch: 03, Loss: 1.2082, Train: 35.77%, Valid: 35.77% Test: 35.77%
Epoch: 04, Loss: 1.2980, Train: 34.62%, Valid: 34.62% Test: 34.62%
Epoch: 05, Loss: 1.2521, Train: 42.05%, Valid: 42.05% Test: 42.05%
Epoch: 06, Loss: 1.2406, Train: 45.48%, Valid: 45.48% Test: 45.48%
Epoch: 07, Loss: 1.1285, Train: 41.85%, Valid: 41.85% Test: 41.85%
Epoch: 08, Loss: 1.1418, Train: 43.57%, Valid: 43.57% Test: 43.57%
Epoch: 09, Loss: 1.1611, Train: 41.11%, Valid: 41.11% Test: 41.11%
Epoch: 10, Loss: 1.2368, Train: 45.30%, Valid: 45.30% Test: 45.30%
Epoch: 11, Loss: 1.2377, Train: 46.82%, Valid: 46.82% Test: 46.82%
Epoch: 12, Loss: 1.0983, Train: 39.12%, Valid: 39.12% Test: 39.12%
Epoch: 13, Loss: 1.1505, Train: 41.93%, Valid: 41.93% Test: 41.93%
Epoch: 14, Loss: 1.3311, Train: 44.37%, Valid: 44.37% Test: 44.37%
Epoch: 15, Loss: 1.1408, Train: 39.28%, Valid: 39.28% Test: 39

In [ ]:
result = test(model, data_loader, [], evaluator)